In [22]:
from collections import Counter 

import pandas as pd
import numpy as np
import re

In [23]:
data_path = "data/"
df = pd.read_csv(data_path + "processed_train.csv")

In [24]:
# df = df[:200]
df

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,"toko dita , kertosono",toko dita/
3,jl. orde baru,/jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede
...,...,...
267998,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya
267999,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani
268000,"raya cila kko , cilandak timur kel.",/raya cila kko
268001,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/


In [25]:
def getPOI(s) :
    return s.split("/")[0]

def getStreet(s) :
    return s.split("/")[1]

df["POI"] = df["POI/street"].apply(getPOI)
df["Street"] = df["POI/street"].apply(getStreet) 

In [26]:
def create_label(s, poi, street) :
    tokens = s.split(" ")
    n = len(tokens)
    tp = poi.split(" ")
    ts = street.split(" ")
    tags = []
    i = 0
    while i < n :
        if len(tp) > 0 and tokens[i] == tp[0] :
            j = 1
            while j < len(tp) :
                if i+j < n and tokens[i+j] == tp[j]:
                    j += 1
                else :
                    break
            if j == len(tp) :
                tags.append("B-POI")
                for k in range(1, j) :
                    tags.append("I-POI")
                i += j
            else :
                tags.append("O")
                i += 1
        elif len(ts) > 0 and tokens[i] == ts[0] :
            j = 1
            while j < len(ts) :
                if i+j < n and tokens[i+j] == ts[j] :
                    j += 1
                else :
                    break
            if j == len(ts) :
                tags.append("B-STREET")
                for k in range(1, j) :
                    tags.append("I-STREET")
                i += j
            else :
                tags.append("O")
                i += 1                
        else :
            tags.append("O")
            i += 1
        
#         print(tokens)
#         print(tags)

    if tokens[-1] != "." :
        tokens.append(".")
        tags.append("O")
    
    res = ""
#     print(tokens)
#     print(tags)
    for token, tag in zip(tokens, tags) :
        res += "{}\t{}\n".format(token, tag)
    return res

In [27]:
s = "raya samb gede , 299 toko bb kids"
poi = "toko bb kids"
street = "raya samb gede"
res = create_label(s, poi, street)
print(res)

raya	B-STREET
samb	I-STREET
gede	I-STREET
,	O
299	O
toko	B-POI
bb	I-POI
kids	I-POI
.	O



In [28]:
df["Label"] = df.apply(lambda x: create_label(x.raw_address, x.POI, x.Street), axis=1)

In [29]:
df

,raw_address,POI/street,POI,Street,Label
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika,jl\tB-STREET\nkapuk\tI-STREET\ntimur\tI-STREET...
1,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung,setu\tO\nsiung\tB-STREET\n119\tO\nrt\tO\n5\tO\...
2,"toko dita , kertosono",toko dita/,toko dita,,"toko\tB-POI\ndita\tI-POI\n,\tO\nkertosono\tO\n..."
3,jl. orde baru,/jl. orde baru,,jl. orde baru,jl.\tB-STREET\norde\tI-STREET\nbaru\tI-STREET\...
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede,raya\tB-STREET\nsamb\tI-STREET\ngede\tI-STREET...
...,...,...,...,...,...
267998,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya,,kakap raya,karawaci\tO\nbaru\tO\nkakap\tB-STREET\nraya\tI...
267999,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,,jend ahmad yani,jend\tB-STREET\nahmad\tI-STREET\nyani\tI-STREE...
268000,"raya cila kko , cilandak timur kel.",/raya cila kko,,raya cila kko,raya\tB-STREET\ncila\tI-STREET\nkko\tI-STREET\...
268001,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,taman asri,,jalan\tO\ncipadu\tO\njaya\tO\ntaman\tB-POI\nas...


In [30]:
final = df.sample(frac=1, random_state=0).reset_index(drop=True)
final

,raw_address,POI/street,POI,Street,Label
0,boj indah ii 27 rt 7 6 pondok kelapa duren sawit,/boj indah ii,,boj indah ii,boj\tB-STREET\nindah\tI-STREET\nii\tI-STREET\n...
1,"kenc v , 23 2 50149 semarang barat",/kenc v,,kenc v,"kenc\tB-STREET\nv\tI-STREET\n,\tO\n23\tO\n2\tO..."
2,"sriyono , beri punduhsari manyaran",sriyono/beri,sriyono,beri,"sriyono\tB-POI\n,\tO\nberi\tB-STREET\npunduhsa..."
3,"joko nugr pengin raya sara , plaosan",/raya sara,,raya sara,joko\tO\nnugr\tO\npengin\tO\nraya\tB-STREET\ns...
4,banyu urip simo gun iii 12 rw 1 60254 sawahan,/simo gun iii,,simo gun iii,banyu\tO\nurip\tO\nsimo\tB-STREET\ngun\tI-STRE...
...,...,...,...,...,...
267998,puncak mulya ranca mulya rt 01 rw 06 des sukaluyu,puncak mulya/,puncak mulya,,puncak\tB-POI\nmulya\tI-POI\nranca\tO\nmulya\t...
267999,r. a. kart lapongkoda tempe,/r. a. kart,,r. a. kart,r.\tB-STREET\na.\tI-STREET\nkart\tI-STREET\nla...
268000,"perumahan green brawijaya no d4 rumah hijau , ...",perumahan green brawijaya/,perumahan green brawijaya,,perumahan\tB-POI\ngreen\tI-POI\nbrawijaya\tI-P...
268001,ban 64 akademi laundry sewon,akademi laundry/ban,akademi laundry,ban,ban\tB-STREET\n64\tO\nakademi\tB-POI\nlaundry\...


In [31]:
n = len(final)
nt = int(0.95 * n)
train = final[:nt+1]
test = final[nt:]

In [32]:
train

,raw_address,POI/street,POI,Street,Label
0,boj indah ii 27 rt 7 6 pondok kelapa duren sawit,/boj indah ii,,boj indah ii,boj\tB-STREET\nindah\tI-STREET\nii\tI-STREET\n...
1,"kenc v , 23 2 50149 semarang barat",/kenc v,,kenc v,"kenc\tB-STREET\nv\tI-STREET\n,\tO\n23\tO\n2\tO..."
2,"sriyono , beri punduhsari manyaran",sriyono/beri,sriyono,beri,"sriyono\tB-POI\n,\tO\nberi\tB-STREET\npunduhsa..."
3,"joko nugr pengin raya sara , plaosan",/raya sara,,raya sara,joko\tO\nnugr\tO\npengin\tO\nraya\tB-STREET\ns...
4,banyu urip simo gun iii 12 rw 1 60254 sawahan,/simo gun iii,,simo gun iii,banyu\tO\nurip\tO\nsimo\tB-STREET\ngun\tI-STRE...
...,...,...,...,...,...
254598,cima 86 66115 kepanjen kidul,/cima,,cima,cima\tB-STREET\n86\tO\n66115\tO\nkepanjen\tO\n...
254599,trape kebon jeruk kebon jeruk,/trape,,trape,trape\tB-STREET\nkebon\tO\njeruk\tO\nkebon\tO\...
254600,slipi v 61 slipi palmerah,/slipi v,,slipi v,slipi\tB-STREET\nv\tI-STREET\n61\tO\nslipi\tO\...
254601,"jampang seta , 35 kemang",/seta,,seta,"jampang\tO\nseta\tB-STREET\n,\tO\n35\tO\nkeman..."


In [33]:
def save_file(filename, arr) :
    f = open(filename, "w+",  encoding='utf-8')
    for s in arr :
        f.write(s.encode("ascii", "ignore").decode())
        f.write("\n")
    f.close()

In [34]:
train_path = data_path + "bert-fine-tune/train.txt"
validation_path = data_path + "bert-fine-tune/validation.txt"

train_data = train["Label"].values.tolist()
test_data = test["Label"].values.tolist()


ntry = None
# ntry = 1000
if ntry :
    train_data = train_data[:ntry]
    test_data = test_data[:ntry]

save_file(train_path, train_data)
save_file(validation_path, test_data)